In [ ]:
---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier


def blight_model():
    #loading datasets
    train=pd.read_csv('train.csv',encoding = 'ISO-8859-1' )
    test=pd.read_csv('test.csv')
           
    #cleaning features from data   
    #droping rows, where label is null, usually due to status "not responsible by disposition"
    train.dropna(axis=0,subset=['compliance'],inplace=True)
    
    #
    
    #transforming text columns to labels
    le=LabelEncoder()
    colnames=['agency_name','inspector_name','violator_name','violation_description','disposition','grafitti_status','violation_code']
    train[colnames]=train[colnames].fillna(value='unknown')
    test[colnames]=test[colnames].fillna(value='unknown')
    traintest=pd.concat([train,test])
    le.fit(trainrtest['agency_name'])
    train['agency_name']=le.transform(train['agency_name'])
    test['agency_name']=le.transform(test['agency_name'])
    le.fit(traintest['inspector_name'])
    train['inspector_name']=le.transform(train['inspector_name'])
    test['inspector_name']=le.transform(test['inspector_name'])
    le.fit(traintest['violator_name'])
    train['violator_name']=le.transform(train['violator_name'])
    test['violator_name']=le.transform(test['violator_name'])
    le.fit(traintest['violation_description'])
    train['violation_description']=le.transform(train['violation_description'])
    test['violation_description']=le.transform(test['violation_description'])
    le.fit(traintest['disposition'])
    train['disposition']=le.transform(train['disposition'])
    test['disposition']=le.transform(test['disposition'])
    le.fit(traintest['grafitti_status'])
    train['grafitti_status']=le.transform(train['grafitti_status'])
    test['grafitti_status']=le.transform(test['grafitti_status'])
    le.fit(traintest['violation_code'])
    train['violation_code']=le.transform(train['violation_code'])
    test['violation_code']=le.transform(test['violation_code'])
    
    #spliting dates to single features
    train['ticket_issued_date']=pd.to_datetime(train['ticket_issued_date']).fillna(method='ffill')
    train['hearing_date']=pd.to_datetime(train['hearing_date']).fillna(method='ffill')
    test['ticket_issued_date']=pd.to_datetime(test['ticket_issued_date']).fillna(method='ffill')
    test['hearing_date']=pd.to_datetime(test['hearing_date']).fillna(method='ffill')
    train['ticket_hour']=train['ticket_issued_date'].apply(lambda x :x.hour)
    train['hearing_hour']=train['hearing_date'].apply(lambda x :x.hour)
    test['ticket_hour']=test['ticket_issued_date'].apply(lambda x :x.hour)
    test['hearing_hour']=test['hearing_date'].apply(lambda x :x.hour)
    train['ticket_issued_date']=train['ticket_issued_date'].apply(lambda x :x.toordinal())
    train['hearing_date']=train['hearing_date'].apply(lambda x :x.toordinal())
    test['ticket_issued_date']=test['ticket_issued_date'].apply(lambda x :x.toordinal())
    test['hearing_date']=test['hearing_date'].apply(lambda x :x.toordinal())
    
    
    #transofrming address to lat lon    
    latlon=pd.read_csv('latlons.csv')
    address=pd.read_csv('addresses.csv')
    latlon.dropna(axis=0,inplace=True)
    addlatlon=address.merge(latlon,how='inner',on='address')
    addlatlon.drop('address',axis=1,inplace=True)
    train.drop(['violation_street_number', 'violation_street_name', 'violation_zip_code'],axis=1,inplace=True)
    test.drop(['violation_street_number', 'violation_street_name', 'violation_zip_code'],axis=1,inplace=True)
    train=train.merge(addlatlon, how='left',on='ticket_id')
    test=test.merge(addlatlon,how='left',on='ticket_id')
    train[['lat','lon']]=train[['lat','lon']].fillna(value=0)
    
    #droping mailing address
    mailing=['mailing_address_str_number', 'mailing_address_str_name', 'city','state', 'zip_code', 'non_us_str_code', 'country']
    train.drop(mailing,axis=1,inplace=True)
    test.drop(mailing,axis=1,inplace=True)
    
    #extracting label from dateset
    y_train=train['compliance']
    #preparing answer series with name and indexes
    answer=pd.Series(index=test['ticket_id'],dtype='float32',name='compliance')
    #droping features unavailable in test data
    train.drop(['payment_amount','payment_date','payment_status','balance_due','collection_status','compliance','compliance_detail'],axis=1,inplace=True)
    
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(train)
    X_test_scaled = scaler.transform(test)
    

    
    
    clf = MLPClassifier(hidden_layer_sizes = [10, 10], alpha = 5.0,
                   random_state = 0, solver='lbfgs').fit(X_train_scaled, y_train)
    proba_test=clf.predict_proba(X_test_scaled)
    answer[:]=proba_test[:,1]
    
    return answer

In [2]:
train=pd.read_csv('train.csv',encoding = 'ISO-8859-1' )
test=pd.read_csv('test.csv')

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#droping rows, where label is null, usually due to status "not responsible by disposition"
train.dropna(axis=0,subset=['compliance'],inplace=True)

In [4]:
#transforming text columns to labels
le=LabelEncoder()
colnames=['agency_name','inspector_name','violator_name','violation_description','disposition','grafitti_status','violation_code']
train[colnames]=train[colnames].fillna(value='unknown')
test[colnames]=test[colnames].fillna(value='unknown')
traintest=pd.concat([train,test])
le.fit(traintest['agency_name'])
train['agency_name']=le.transform(train['agency_name'])
test['agency_name']=le.transform(test['agency_name'])
le.fit(traintest['inspector_name'])
train['inspector_name']=le.transform(train['inspector_name'])
test['inspector_name']=le.transform(test['inspector_name'])
le.fit(traintest['violator_name'])
train['violator_name']=le.transform(train['violator_name'])
test['violator_name']=le.transform(test['violator_name'])
le.fit(traintest['violation_description'])
train['violation_description']=le.transform(train['violation_description'])
test['violation_description']=le.transform(test['violation_description'])
le.fit(traintest['disposition'])
train['disposition']=le.transform(train['disposition'])
test['disposition']=le.transform(test['disposition'])
le.fit(traintest['grafitti_status'])
train['grafitti_status']=le.transform(train['grafitti_status'])
test['grafitti_status']=le.transform(test['grafitti_status'])
le.fit(traintest['violation_code'])
train['violation_code']=le.transform(train['violation_code'])
test['violation_code']=le.transform(test['violation_code'])

In [ ]:
test.head(3)

In [ ]:
train.head(3)

In [5]:
#spliting dates to single features
train['ticket_issued_date']=pd.to_datetime(train['ticket_issued_date']).fillna(method='ffill')
train['hearing_date']=pd.to_datetime(train['hearing_date']).fillna(method='ffill')
test['ticket_issued_date']=pd.to_datetime(test['ticket_issued_date']).fillna(method='ffill')
test['hearing_date']=pd.to_datetime(test['hearing_date']).fillna(method='ffill')
train['ticket_hour']=train['ticket_issued_date'].apply(lambda x :x.hour)
train['hearing_hour']=train['hearing_date'].apply(lambda x :x.hour)
test['ticket_hour']=test['ticket_issued_date'].apply(lambda x :x.hour)
test['hearing_hour']=test['hearing_date'].apply(lambda x :x.hour)
train['ticket_issued_date']=train['ticket_issued_date'].apply(lambda x :x.toordinal())
train['hearing_date']=train['hearing_date'].apply(lambda x :x.toordinal())
test['ticket_issued_date']=test['ticket_issued_date'].apply(lambda x :x.toordinal())
test['hearing_date']=test['hearing_date'].apply(lambda x :x.toordinal())

In [6]:
latlon=pd.read_csv('latlons.csv')
address=pd.read_csv('addresses.csv')
latlon.dropna(axis=0,inplace=True)
addlatlon=address.merge(latlon,how='inner',on='address')
addlatlon.drop('address',axis=1,inplace=True)
train.drop(['violation_street_number', 'violation_street_name', 'violation_zip_code'],axis=1,inplace=True)
test.drop(['violation_street_number', 'violation_street_name', 'violation_zip_code'],axis=1,inplace=True)
train=train.merge(addlatlon, how='left',on='ticket_id')
test=test.merge(addlatlon,how='left',on='ticket_id')
train[['lat','lon']]=train[['lat','lon']].fillna(value=0)
test[['lat','lon']]=test[['lat','lon']].fillna(value=0)

In [7]:
mailing=['mailing_address_str_number', 'mailing_address_str_name', 'city','state', 'zip_code', 'non_us_str_code', 'country']
train.drop(mailing,axis=1,inplace=True)
test.drop(mailing,axis=1,inplace=True)

In [8]:
#extracting label from dateset
y_train=train['compliance']
#preparing answer series with name and indexes
answer=pd.Series(index=test['ticket_id'],dtype='float32',name='compliance')
#droping features unavailable in test data
train.drop(['payment_amount','payment_date','payment_status','balance_due','collection_status','compliance','compliance_detail'],axis=1,inplace=True)
    

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train)
X_test_scaled = scaler.transform(test)

In [10]:
clf = MLPClassifier(hidden_layer_sizes = [10, 10], alpha = 5.0,
                   random_state = 0, solver='lbfgs').fit(X_train_scaled, y_train)

In [36]:
proba_test=clf.predict_proba(X_test_scaled)

In [33]:
answer=pd.Series(index=test['ticket_id'],dtype='float32',name='compliance')

In [28]:
clf.classes_

array([ 0.,  1.])

In [37]:
answer[:]=proba_test[:,1]

In [38]:
answer.head(5)

ticket_id
284932    0.068890
285362    0.011227
285361    0.065141
285338    0.033121
285346    0.097434
Name: compliance, dtype: float64

In [ ]:
test[['lat','lon']]=test[['lat','lon']].fillna(value=0)

In [ ]:
test.isnull().any()

In [ ]:
train.dtypes

In [ ]:
address.head(5)

In [ ]:
address.loc[address['address']=='2900 tyler, Detroit MI']

In [ ]:
address.loc[address['address']=='2900 tyler, Detroit MI']['ticket_id'].as_matrix()

In [ ]:
address.loc[address['ticket_id']==285530 ]['address'].iloc[0]

In [ ]:
lat=latlon.loc[latlon['address']=='2900 tyler, Detroit MI']['lat'].iloc[0]
lat

In [ ]:
latlon.head(5)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
pca = PCA(n_components=1)
latlon.dropna(axis=0,inplace=True)
scaler = StandardScaler()

X_PCA=latlon[['lat','lon']].as_matrix()
X_PCA_scaled = scaler.fit_transform(X_PCA)
llpca=pca.fit_transform(X_PCA)
llpca

In [ ]:
addlatlon=address.merge(latlon,how='inner',on='address')

In [ ]:
addlatlon.head(5)

In [ ]:
train[train['ticket_id']==28204]

In [ ]:
train.head(5)

In [ ]:
train.dropna(axis=0,subset=['compliance']).head(5)

In [ ]:
train[train['grafitti_status'].notnull()]

In [ ]:
test=pd.read_csv('test.csv')

In [ ]:
test.head(10)

In [ ]:
train.drop(['violation_street_number', 'violation_street_name', 'violation_zip_code'],axis=1,inplace=True)
train=train.merge(addlatlon, how='left',on='ticket_id')
train.head(5)

In [ ]:
le=LabelEncoder()
train['agency_name']=le.fit_transform(train['agency_name'])
train.head(5)

In [ ]:
train['agency_name'].unique()

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train['violator_name']=train[train['violator_name'].isnull()]['violator_name']='unknown'

In [ ]:
train[train['violator_name'].isnull()]

In [ ]:
colnames=['agency_name','inspector_name','violator_name','violation_description','disposition','grafitti_status']
train[colnames]=train[colnames].fillna(value='unknown')
test[colnames]=test[colnames].fillna(value='unknown')

In [ ]:
train[train['grafitti_status'].isnull()]

In [ ]:
int(str(test['ticket_issued_date'].iloc[0]).split()[1].split(sep=":")[0])

In [ ]:
traintest['judgment_amount'].isnull().any()